In [ ]:
#This web scraping script is from Syed Sadat Nazrul
#https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059

In [1]:
import sys
!{sys.executable} -m pip install lxml

     |████████████████████████████████| 5.5MB 18.9MB/s eta 0:00:01


In [2]:
import requests
import lxml.html as lh
import pandas as pd

In [3]:
url='https://data.j-league.or.jp/SFMS01/search?team_ids=33&home_away_select=0'
#Create a handle, page, to handle the contents of the website
page = requests.get(url, cookies={'SFCM01LANG':'en'})

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [7]:
page.headers


{'Content-Type': 'text/html;charset=UTF-8', 'Date': 'Mon, 19 Oct 2020 00:57:11 GMT', 'Server': 'nginx', 'Set-Cookie': 'JSESSIONID=5459560AFC6190F67445CDA4573BB0A4; Path=/; Secure; HttpOnly, SFCM01LANG=en; Max-Age=63072000; Expires=Wed, 19-Oct-2022 00:57:11 GMT; Path=/, login=0; Path=/, login=0; Path=/, AWSELB=99A9D30F06BEECDEECB58B4E6E98D4F19DDE93F7F3359158DF2277340366A98F8401D7893BFA00B9E23DF21BCA686C92B4BC7F77BF1554E185B36E052517CCB639AAC48286901CC226FBD628174BA6BE7498E005B0;PATH=/;SECURE;HTTPONLY', 'X-Frame-Options': 'SAMEORIGIN', 'X-XSS-Protection': '1; mode=block', 'transfer-encoding': 'chunked', 'Connection': 'keep-alive'}

In [4]:
#Check the length of the first 12 rows
print([len(T) for T in tr_elements[:12]])

TEXT = '<br>test&nbsp;&nbsp;test'
print(TEXT.replace('<br>', '').replace('&nbsp;', ''))

[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
testtest


In [5]:
tr_elements = doc.xpath('//tr')

#Create empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print("{}".format(name))
    col.append((name,[]))

Year
Tournaments
Sec
Date
Kick-off
Home
Score
Away
Venue
Att.


In [14]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=10:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content().replace('&nbsp', '').strip()
        #print(data)
        
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
            
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [15]:


#checking length of each column
[len(C) for (title,C) in col]

[2892, 2892, 2892, 2892, 2892, 2892, 2892, 2892, 2892, 2892]

In [16]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [17]:
df.to_csv('All_Tosu.csv')